In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import pandas as pd
import requests
import re
from lxml import etree
from pyproj import CRS
#爬取数据
china = gpd.read_file('/Users/jocelynli/Library/CloudStorage/OneDrive-stu.ecnu.edu.cn/ECNU/大三上/地理信息系统底层开发/期末作业/data/China/China.shp')
pr=china['省份'].tolist()
file_list=[] #用于存储爬取的数据名称
pr_affected=[] #用于存储每次台风影响的省
url = "http://agora.ex.nii.ac.jp/cgi-bin/dt/search_name2.pl?lang=en&basin=wnp&smp=1&sdp=1&emp=12&edp=31"
response = requests.get(url)
response.encoding = 'utf8'
html = response.text
d = re.findall(r'<td><a href="/digital-typhoon/summary/wnp/s/(.*?)">',html,re.S) # 搜索所有的台风信息
for p in d: #遍历所有的台风网页
    id = p[:6]  #获取台风的ID
    if int(id) >= 200001: #筛选出2000-的所有台风
        p_url = 'http://agora.ex.nii.ac.jp/digital-typhoon/summary/wnp/k/' + id  # 台风轨迹信息
        response_1 = requests.get(p_url)
        html_1 = etree.HTML(response_1.text)
        table = html_1 .xpath('//table[@class="TRACKINFO"]')
        table = etree.tostring(table[0], encoding='utf-8').decode()
        df = pd.read_html(table, encoding='utf-8', header=0)[0]
        df = df.iloc[:, :6] #删除无用列
        column=['Year','Month','Day','Hour','Lat','Lon'] #添加列名
        df.columns=column
        df['Id'] = id
        points = [Point(lon, lat) for lon, lat in zip(df['Lon'], df['Lat'])] #获取该次台风的各时间台风中心的经纬度数据
        buffered_points = [point.buffer(300/111) for point in points] #设置缓冲区，缓冲区半径为台风7级风圈300km，由于坐标系为EPSG:4326，所以buffer半径的单位为度，1度约为111km
        geometries = GeoDataFrame(geometry=buffered_points)
        geometries.set_crs('EPSG:4326', inplace=True)
        intersected = gpd.overlay(geometries, china, how='intersection') #获取台风影响范围和江浙沪城市的重合部分
        if not intersected.empty: #若台风7级风圈与研究区有重合范围，则保存该台风数据
            df.to_excel(f'/Users/jocelynli/Library/CloudStorage/OneDrive-stu.ecnu.edu.cn/ECNU/大三上/地理信息系统底层开发/期末作业/data/台风excel/{id}.xlsx', index = False)
            geometry = [Point(lon, lat) for lon, lat in zip(df['Lon'], df['Lat'])]
            gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
            gdf.to_file(f'/Users/jocelynli/Library/CloudStorage/OneDrive-stu.ecnu.edu.cn/ECNU/大三上/地理信息系统底层开发/期末作业/data/台风shp/{id}.shp', driver='ESRI Shapefile')
            print(f'{id} Done.')
            affected_pr = list(set(intersected['省份'].tolist())) #获取受到该台风影响的省份，删除其中的重复值
            pr_affected.append(affected_pr)
    counts = {} #用于统计每个城市10年间过境的台风个数
    for m in pr_affected:
        for n in m:
            if n in counts:
                counts[n] += 1
            else:
                counts[n] = 1
    counts = dict(sorted(counts.items(), key=lambda x: x[0]))